In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml

In [ ]:
# TO DO:
# add gradient clipping?

# TO TEST:
# enforcing norm constraint on P
# how to do exploration
# whether this works at all?

class klqr:
    # not currently doing value updates at varying rates
    # not currently doing double Q learning (what would this look like?)
    
    def __init__(self,config,sess):
        self.sess = sess
        
        self.x_dim = config['x_dim']
        self.z_dim = config['z_dim']
        self.a_dim = config['a_dim']
        self.lr = config['lr']
        self.horizon = config['horizon']
        self.gamma = config['discount_rate']

        
        ou_theta = config['ou_theta']
        ou_sigma = config['ou_sigma']
        self.config = config
        
        # Ornstein-Uhlenbeck noise for exploration -- code from Yuke Zhu
        self.noise_var = tf.Variable(tf.zeros([self.a_dim,1]))
        noise_random = tf.random_normal([self.a_dim,1], stddev=ou_sigma)
        self.noise = self.noise_var.assign_sub((ou_theta) * self.noise_var - noise_random)

        self.max_riccati_updates = config['max_riccati_updates']
        self.train_batch_size = config['train_batch_size']
        self.replay_buffer = ReplayBuffer(buffer_size=config['replay_buffer_size'])
        
        self.experience_count = 0
        
    def build_model(self):        

        with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
            
            self.x_ = tf.placeholder(tf.float32,shape=[None, self.x_dim])
            self.xp_ = tf.placeholder(tf.float32,shape=[None, self.x_dim])
            self.a_ = tf.placeholder(tf.float32,shape=[None, self.a_dim])
            self.r_ = tf.placeholder(tf.float32,shape=[None])

            self.z = self.encoder(self.x_)
            self.zp = self.encoder(self.xp_)

            print('z shape:', self.z.get_shape())

            #init R

            self.R_asym = tf.get_variable('R_asym',shape=[self.a_dim,self.a_dim])
    #         self.R_asym = tf.Variable(np.random.rand(self.a_dim,self.a_dim) - 0.5)

            # working with Ra.T Ra so that inner product is norm(Rx) and not norm(R.T x)
            self.R = tf.matmul(tf.transpose(self.R_asym),self.R_asym)

            #init Q -- shape: z_dim * z_dim
            self.Q_asym = tf.get_variable('Q_asym',shape=[self.z_dim,self.z_dim])
            self.Q = tf.matmul(tf.transpose(self.Q_asym),self.Q_asym)

            #init P -- shape: z_dim * z_dim
            self.P = tf.get_variable('P_asym',shape=[self.z_dim,self.z_dim],trainable=False,initializer=tf.initializers.identity)
            self.P_asym = tf.transpose(tf.cholesky(self.P)) #this might need to have the transpose removed?

            #init B -- shape: z_dim * u_dim
            self.B = tf.get_variable('B',shape=[self.z_dim,self.a_dim])
    #         self.B = tf.Variable(np.random.rand(self.z_dim,self.u_dim) - 0.5)

            #init A -- shape: z_dim * z_dim
            self.A = tf.get_variable('A',shape=[self.z_dim,self.z_dim])
    #         self.A = tf.Variable(np.random.rand(self.z_dim,self.z_dim) - 0.5)

            #define K -- shape: u_dim * z_dim
            term1 = tf.matrix_inverse(self.R + tf.matmul(tf.matmul(tf.transpose(self.B),self.Q),self.B))
            term2 = tf.matmul(tf.matmul(tf.transpose(self.B),self.P),self.A)
            self.K = tf.matmul(term1,term2)
            self.policy_action = tf.transpose(tf.matmul(self.K,tf.transpose(self.z)))

            #make reward negative to convert to cost
            self.bootstrapped_value = -self.r_ + self.gamma*tf.square(tf.norm(tf.transpose(tf.matmul(self.P_asym,tf.transpose(self.zp))),axis=1))

            action_cost = tf.square(tf.norm(tf.transpose(tf.matmul(self.R_asym,tf.transpose(self.a_))),axis=1))#can simplify this by taking norm on other axis
            state_cost = tf.square(tf.norm(tf.transpose(tf.matmul(self.Q_asym,tf.transpose(self.z))),axis=1)) 
            self.PABK = tf.matmul(self.P_asym, self.A + tf.matmul(self.B,self.K))
            Vzp = tf.square(tf.norm(tf.transpose(tf.matmul(self.PABK,tf.transpose(self.zp))),axis=1))
            self.Qsa = action_cost + state_cost + Vzp

            self.td_loss = tf.reduce_mean(self.bootstrapped_value - self.Qsa)
            #can add regularization via P, dynamics, sparsity, etc
            self.loss = self.td_loss 
            global_step = tf.Variable(0, trainable=False, name='global_step')
            optimizer = tf.train.AdamOptimizer(self.lr)
            self.train_op = optimizer.minimize(self.loss, global_step=global_step)
    
            self.sess.run(tf.global_variables_initializer())

    
    def update_model(self):        
        #this function is mostly taken from Yuke's code
        print('updating model')
        if self.replay_buffer.count() < self.train_batch_size:
            return
        
        batch           = self.replay_buffer.getBatch(self.train_batch_size)
        
        states          = np.zeros((self.train_batch_size, self.x_dim))
        rewards         = np.zeros((self.train_batch_size))
        actions         = np.zeros((self.train_batch_size, self.a_dim))
        next_states     = np.zeros((self.train_batch_size, self.x_dim))

        for k, (s0, a, r, s1, done) in enumerate(batch):
            #currently throwing away done states; should fix this
            states[k] = s0
            rewards[k] = r
            actions[k] = a
            next_states[k] = s1
            # check terminal state
#             if not done:
#                 next_states[k] = s1
#                 next_state_mask[k] = 1

        cost, _ = self.sess.run([self.loss, self.train_op],
        {
        self.x_:  states,
        self.xp_: next_states,
        self.a_:  actions,
        self.r_:  rewards
        })
    
        #possibly update target via Riccati recursion? or do standard target separation? 
    
    def update_P(self):
        print('updating P')
        for k in range(self.max_riccati_updates):
            #do Riccati backup in tensorflow oh god why
            ABK = self.A + tf.matmul(self.B,self.K)
            APA = tf.matmul(tf.matmul(tf.transpose(ABK),self.P),ABK) #
            self.P = self.Q + tf.matmul(tf.matmul(tf.transpose(self.K),self.R),self.K) + self.gamma*APA
        
        self.P_asym = tf.transpose(tf.cholesky(self.P))
        print(sess.run(self.P))
            #TODO add a termination criterion for norm of Riccati update difference?
        
    def pi(self,x,explore=True):
        self.experience_count += 1
        x = np.reshape(x,(1,3))
        
        a,w = self.sess.run([self.policy_action,self.noise], {self.x_: x})
        
        a = a + w if explore else a
        # TODO check the dimension of the output of this
        return [a[0,0]]
        
    def store_experience(self,s,a,r,sp,done):
        # currently storing experience for every iteration
        self.replay_buffer.add(s, a, r, sp, done)
    
    def encoder(self,x,name="encoder",batch_norm=False):
        layer_sizes = self.config['encoder_layers']
        with tf.variable_scope(name,reuse=tf.AUTO_REUSE):
            inp = x
            for units in layer_sizes: 
                inp = tf.layers.dense(inputs=inp, units=units,activation=tf.nn.relu)

            z = tf.layers.dense(inputs=inp, units=self.z_dim,activation=None)

        if batch_norm:
            z = tf.layers.batch_normalization(z)

        return z

class ReplayBuffer:
    # taken from Yuke Zhu's Q learning implementation
    
    def __init__(self, buffer_size):

        self.buffer_size = buffer_size
        self.num_experiences = 0
        self.buffer = deque()

    def getBatch(self, batch_size):
        # random draw N
        return random.sample(self.buffer, batch_size)

    def size(self):
        return self.buffer_size

    def add(self, state, action, reward, next_action, done):
        new_experience = (state, action, reward, next_action, done)
        if self.num_experiences < self.buffer_size:
          self.buffer.append(new_experience)
          self.num_experiences += 1
        else:
          self.buffer.popleft()
          self.buffer.append(new_experience)

    def count(self):
        # if buffer is full, return buffer size
        # otherwise, return experience counter
        return self.num_experiences

    def erase(self):
        self.buffer = deque()
        self.num_experiences = 0

In [ ]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
# simulates the agent acting in env, yielding every N steps
# (decouples episode reseting mechanics from the training alg)
def experience_generator(agent, env, N):
    s = env.reset()
    n_steps = 0
    n_eps = 0
    last_cum_rew = 0
    cum_rew = 0
    while True:
        n_steps += 1
        a = agent.pi(s)
        sp, r, done,_ = env.step(a)
        cum_rew += r
        if done:
            n_eps += 1
            last_cum_rew = cum_rew
            cum_rew = 0
            s = env.reset()
        else:
            agent.store_experience(s, a, r, sp, done)
            s = sp

        if n_steps % N == 0:
            yield (n_steps, n_eps, last_cum_rew)



def train_agent(agent, env,
                max_timesteps=0, max_episodes=0, max_iters=0, max_seconds=0, # time constraint
                n_transitions_between_updates=100,
                n_optim_steps_per_update=100,
                n_optim_steps_per_p_update=100,
                ):

    # run an episode, and feed data to model
    episodes_so_far = 0
    timesteps_so_far = 0
    iters_so_far = 0
    tstart = time.time()

    assert sum([max_iters>0, max_timesteps>0, max_episodes>0, max_seconds>0])==1, "Only one time constraint permitted"

    exp_gen = experience_generator(agent, env, n_transitions_between_updates)

    while True:
        iters_so_far += 1
        if max_timesteps and timesteps_so_far >= max_timesteps:
            break
        elif max_episodes and episodes_so_far >= max_episodes:
            break
        elif max_iters and iters_so_far >= max_iters:
            break
        elif max_seconds and time.time() - tstart >= max_seconds:
            break

        print("********** Iteration %i ************"%iters_so_far)

        # gather experience
        episodes_so_far, timesteps_so_far, last_cum_rew = exp_gen.__next__()

        # optimize the model from collected data:
        for i in range(n_optim_steps_per_update):
            agent.update_model()

            if (i+1) % n_optim_steps_per_p_update == 0:
                agent.update_P()

        print("\tLast Episode Reward: %d"%last_cum_rew)
        # add other logging stuff here
        # add saving checkpoints here


In [ ]:
env = gym.make('Pendulum-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=100)